# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Открытие файла и изучение общей информации

In [1]:
# Импортируем Pandas и читаем файл с данными
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# Получаем общую информацию о файле и выводим первые 10 строк для наглядности
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В таблице 12 столбцов. Тип данных столбцов разный: float64(2), int64(5), object(5).

Столбцы содержат информацию:

* children - количество детей в семье;
* days_employed - общий трудовой стаж в днях;
* dob_years - возраст клиента в годах;
* education - уровень образования клиента;
* education_id - идентификатор уровня образования;
* family_status - семейное положение;
* family_status_id - идентификатор семейного положения;
* gender - пол клиента;
* income_type - тип занятости;
* debt - имел ли задолженность по возврату кредитов;
* total_income - ежемесячный доход;
* purpose - цель получения кредита.

В столбцах days_employed и total_income количество значений меньше чем у других. Следовательно в данных имеются NaN значения. В столбце days_employed обнаружено много отрицательных значений, а так же трудно обьяснимые значения: как например "340266.072047" соответствует 932 годам трудового стажа. Значения в столбце education написаны в разном регистре.

**Вывод**

Отрицательные значения days_employed можно сделать положительными. О завышенных значениях days_employed было бы неплохо уточнить у заказчиков. Но все же главная проблема - пропуски. В столбце days_employed можно их заменить на 0 так как данный параметр не учавствует в расчетах зависимости которые хотят заказчики. Пропуски в столбце total_income логичнее будет заполнить медианой.

## Предобработка данных

### Обработка пропусков

In [2]:
# Считаем суммарное количество пропусков
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Одинаковое колличество пропусков в столбцах days_employed и total_income. Причины могу крыться в неправильноый выгрузке данных.

In [3]:
# Сделаем функцию перевода отрицательных значений в положительные
def reverse(days):
    if days < 0:
        days *= -1
        return days
    return days
# Заменяем отрицательные значения в days_employed на положительные
data['days_employed'] = data['days_employed'].apply(reverse)
# Заменяем пропуски в days_employed на 0
data['days_employed'] = data['days_employed'].fillna(0)
# Находим медиану total_income для каждого типа занятости
medians_total_income = data.groupby('income_type')['total_income'].median()
medians_total_income
# Замена медиан на пустые значения
for inc_type in medians_total_income.index:
    data.loc[data['income_type'] == inc_type,'total_income'] = data.loc[data['income_type'] == inc_type,'total_income'].fillna(medians_total_income[inc_type])
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Убрали пустые значения в столбцах days_employed и total_income.

### Замена типа данных

In [4]:
#Замена float на int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Для увеличения точности и избежания неявных ошибок заменили тип данных float на int

### Обработка дубликатов

In [5]:
#Приведем в единый регистр столбец education
data['education'] = data['education'].str.lower()
data['education'].value_counts()
#Проверим уникальные строки в столбце family_status
data['family_status'].value_counts()
#Проверим уникальные строки в столбце gender
data['gender'].value_counts()
data[data.loc[:, 'gender'] == 'XNA']
#Удаляем строку с полом XNA из data
data.drop(data[data['gender'] == 'XNA'].index, inplace=True)
#Проверим уникальные строки в income_type
data['income_type'].value_counts()
#Проверим уникальные строки в purpose
data['purpose'].value_counts()
#Проверим уникальные строки в children
data['children'].value_counts()
#Заменяем значение -1 на 1
#20 детей то же выбивается из общей картины. Предположим, что случайно вместо 2 записали 20
#Заменим значение 20 на 2
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()
#Поиск полных дубликатов и удаление их
data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop = True)
#Проверка
data.duplicated().sum()

0

**Вывод**

Привели к одному регистру Столбец education, удалили строчку с неизвестным полом XNA.Так же было отрицательное количество детей и очень большое значение в 20 детей. Эти строки были откорректированы. Удалили полные дубликаты.

### Лемматизация

In [6]:
#Импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()
#Посмотрим уникальные строки целей займа
data['purpose'].value_counts()
#Создадим в ручную список ключевых слов займа
#Проводим Лемматизацию и замену лемм на ключевое слово
purpose_list = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']
def lemmatize_text(text):
    lemma = m.lemmatize(text)
    for word in purpose_list:
        if word in lemma:
            lemma = word
    return lemma

#Добавим новый столбец с группой по целям займа
data['purpose_group'] = data['purpose'].apply(lemmatize_text)    
data['purpose_group'].value_counts()

#Жилье логично будет обьеденить с недвижимостью
data.loc[data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
data['purpose_group'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

**Вывод**

Из целей кредита выделили вручную ключевые слова и обьеденили похожие по смыслу цели займа в группы. Также цели связанные с покупкой жилья мы обьеденили с целями связанными с покупкой недвижимости. В итоге мы получили 4 больших группы целей займа: недвижимость, автомобиль, образование и свадьба.

### Категоризация данных

In [7]:
#Деление на группы по целем займа своего рода то же категоризация целей. Поэтому Цели уже не трогаем.

#Выделим группы по колличеству детей
# нет детей (0)
# 1-2 ребенка
# многодетные (>3)
def children_group(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
data['children_group'] = data.apply(children_group, axis=1)

#Выделяем группы по заработку
#Разбитие на квантилии
quantile = data['total_income'].quantile([0.25, 0.50, 0.75]).array
quantile
def income_group(income):
    if income <= quantile[0]:
        return 1
    elif (quantile[0] < income) and (income <= quantile[1]):
        return 2
    elif (quantile[1] < income) and (income <= quantile[2]):
        return 3
    else:
        return 4
data['income_group'] = data['total_income'].apply(income_group)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1-2 ребенка,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1-2 ребенка,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,1-2 ребенка,4
21449,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,нет детей,3
21450,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,1-2 ребенка,1
21451,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,многодетные,4


**Вывод**

Провели категоризацию по целям для займа, количеству детей и ежемесячному доходу. Можно было бы провести категоризацию по трудовому стажу и возрасту заемщика, но данные группы не нужны для ответа на поставленные задачи заказчика.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
#Строим сводную таблицу по кол-ву детей и имеющейся задолженности.
data_pivot = data.pivot_table(index = ['children_group'], values = 'debt').sort_values(by = 'debt', ascending = False)
data_pivot

,debt
children_group,
1-2 ребенка,0.092654
многодетные,0.081579
нет детей,0.075444


**Вывод**

Судя по результатам заемщики не имеющие детей менее склонны к просрочке по займу.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#Строим сводную таблицу по семейному положению и имеющейся задолженности.
data_pivot2 = data.pivot_table(index = ['family_status', 'family_status_id'], values='debt').sort_values(by = 'debt', ascending = False)
data_pivot2

,,debt
family_status,family_status_id,
Не женат / не замужем,4,0.097509
гражданский брак,1,0.093494
женат / замужем,0,0.075452
в разводе,3,0.071130
вдовец / вдова,2,0.065693


**Вывод**

Граждане которые ни разу не были в браке или не имеют узаконенных отношений более склонны к просрочке платежей по займу.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
#Строим сводную таблицу по уровню дохода и имеющейся задолженности.
data_pivot3 = data.pivot_table(index = ['income_group'], values='debt').sort_values(by = 'debt', ascending = False)
data_pivot3

,debt
income_group,
2,0.088155
3,0.085382
1,0.079605
4,0.071415


**Вывод**

Судя по результатам, клиет который зарабатывает больше чем 75% выборки с большей вероятностью вернет займ в срок, что в прочем ожидаемо. Но вот на втором месте к удивлению оказалась группа не 50-75% выборки, а группа с наименьшим доходом (<25%). Это можно обьяснить тем, что люди с наименьшим доходом больше контролируют свои расходы и бюджет.

Шанс задолженности клиента:
* доход от 107,6 до 142,6 тыс. рублей в месяц = 8,8%
* доход от 142,6 до 195,8 тыс. рублей = 8,5%
* доход меньше или равен 107,6 тыс. руб. = 8,0%
* больше 195,8 тыс. рублей = 7,1%

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
#Строим сводную таблицу по целям кредита и имеющейся задолженности.
data_pivot4 = data.pivot_table(index = ['purpose_group'], values='debt').sort_values(by = 'debt', ascending = False)
data_pivot4

,debt
purpose_group,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.072340


**Вывод**

Граждане берущие заем связанный с приобретением или ремонтом недвижимости более склонны к выплатам его в срок.

## Общий вывод

На поставленный вопрос "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?" можно однозначно ответить: да, влияет.
Более ответственными являются клиенты состоящие в официальном браке или хотябы раз бывшие в нем и не имеющие детей.
Менее ответственными можно считать граждан которые никогда не состояли в официальном браке и при этом имеют 1 или 2 детей.
